## Notebook for training

To make sure all imports work correctly export PYTHONPATH leading to your Mask RCNN directory.

In [ ]:
# TODO multi GPU training

In [1]:
import os
import sys

ROOT_DIR = os.path.abspath("../")

from imgaug import augmenters as iaa
import numpy as np

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config
from datasets import MultiClassCellDataset

Using TensorFlow backend.


## Load dataset

Check if your data are in expected format and visualize sample annotations.
Dataset structure:


In [2]:
train_data_dir = './sample_dataset'
test_data_dir = './sample_dataset'

train_dataset = MultiClassCellDataset()
train_dataset.init_dataset(train_data_dir)

test_dataset = MultiClassCellDataset()
test_dataset.init_dataset(test_data_dir)

In [3]:
train_dataset.class_from_source_map

{'': 0, 'boundary': 1, 'cell': 2}

## Configuration

Training parameters for model.

In [4]:
class cellConfig(Config):
    NAME = "cell"

    # Adjust to GPU memory
    IMAGES_PER_GPU = 1

    NUM_CLASSES = 2  # Background + cell

    STEPS_PER_EPOCH = 45
    VALIDATION_STEPS = 5

    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between nucleus and BG
    DETECTION_MIN_CONFIDENCE = 0.5

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    BACKBONE = "resnet50"

    # Input image resizing
    # Random crops of size 512x512
    IMAGE_RESIZE_MODE = "crop"
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    IMAGE_MIN_SCALE = 2.0

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    # RPN_NMS_THRESHOLD = 0.9
    RPN_NMS_THRESHOLD = 0.99

    # How many anchors per image to use for RPN training
    # RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    RPN_TRAIN_ANCHORS_PER_IMAGE = 128

    # Image mean (RGB)
    # MEAN_PIXEL = np.array([43.53, 39.56, 48.22])
    MEAN_PIXEL = np.array([126, 126, 126])
    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    USE_MINI_MASK = True
    # MINI_MASK_SHAPE = (56, 56)  # (height, width) of the mini-mask
    MINI_MASK_SHAPE = (100, 100)

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    # TRAIN_ROIS_PER_IMAGE = 128
    TRAIN_ROIS_PER_IMAGE = 256

    # Maximum number of ground truth instances to use in one image
    # MAX_GT_INSTANCES = 200
    MAX_GT_INSTANCES = 500
    # Max number of final detections per image
    # DETECTION_MAX_INSTANCES = 400
    DETECTION_MAX_INSTANCES = 1000

## Training

In [5]:
DEVICE = "/cpu:0" 

def get_model(mode, model_dir, config):
    with tf.device(DEVICE):
        model = modellib.MaskRCNN(mode='',
                                  model_dir=model_dir,
                                  config=config)
    return model

In [6]:
def train(model, config, train_dataset, val_dataset):
    print(train_dataset.class_from_source_map)
    print(val_dataset.class_from_source_map)
    
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.OneOf([iaa.Affine(rotate=90),
                   iaa.Affine(rotate=180),
                   iaa.Affine(rotate=270)]),
        iaa.Multiply((0.8, 1.5)),
        iaa.GaussianBlur(sigma=(0.0, 5.0))
        # iaa.GaussianBlur(sigma=(0,3.0))
    ])

    print("Train network heads")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE,
                # learning_rate = 0.01,
                epochs=100,
                augmentation=augmentation,
                layers='heads')

    print("Train all layers")
    model.train(train_dataset, val_dataset,
                learning_rate=config.LEARNING_RATE,
                # learning_rate = 0.1,
                epochs=200,
                augmentation=augmentation,
                layers='all')

In [7]:
# define the path of pretrained weight to start from
weights_path = None

out_dir = './training_test'
print('> Training on data: ', train_data_dir)
print('> Saving results to: ', out_dir)

config = cellConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=out_dir)

if weights_path is not None:
    print('> Loading weights from: ', weights_path)
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"
    ])

train(model, config, train_dataset, test_dataset)


> Training on data:  ./sample_dataset
> Saving results to:  ./training_test
Instructions for updating:
Colocations handled automatically by placer.
{'': 0, 'boundary': 1, 'cell': 2}
{'': 0, 'boundary': 1, 'cell': 2}
Train network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: ./training_test/cell20190418T1727/mask_rcnn_cell_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistrib

/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/100
image_info {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'annotations': ['./sample_dataset/annotations/sample_21/cell.png', './sample_dataset/annotations/sample_21/boundary.png']}
image_info {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'annotations': ['./sample_dataset/annotations/sample_21/cell.png', './sample_dataset/annotations/sample_21/boundary.png']}
class_id {'': 0, 'boundary': 1, 'cell': 2}
image_info {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'annotations': ['./sample_dataset/annotations/sample_21/cell.png', './sample_dataset/annotations/sample_21/boundary.png']}
image_info {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'annotations': ['./sample_dataset/annotations/sample_21/cell.png', './sample_dataset/annotations/sample_21/boundary.png']}
image_info {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'an

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(p

MemoryErrorERROR:root:Error processing image {'id': 4, 'source': 'custom', 'path': './sample_dataset/images/sample_21.jpg', 'annotations': ['./sample_dataset/annotations/sample_21/cell.png', './sample_dataset/annotations/sample_21/boundary.png']}
Traceback (most recent call last):
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/model.py", line 1695, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/model.py", line 1218, in load_image_gt
    mask = utils.resize_mask(mask, scale, padding, crop)
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/utils.py", line 514, in resize_mask
    mask = scipy.ndimage.zoom(mask, zoom=[scale, scale, 1], order=0)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 615, in zoom
    shape=output_shape)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/scipy/ndimage/_ni_support.p

...

image_info {'id': 41, 'source': 'custom', 'path': './sample_dataset/images/sample_6.jpg', 'annotations': ['./sample_dataset/annotations/sample_6/cell.png', './sample_dataset/annotations/sample_6/boundary.png']}
class_id {'': 0, 'boundary': 1, 'cell': 2}


ERROR:root:Error processing image {'id': 27, 'source': 'custom', 'path': './sample_dataset/images/sample_16.jpg', 'annotations': ['./sample_dataset/annotations/sample_16/cell.png', './sample_dataset/annotations/sample_16/boundary.png']}
Traceback (most recent call last):
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/model.py", line 1695, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/model.py", line 1218, in load_image_gt
    mask = utils.resize_mask(mask, scale, padding, crop)
  File "/data/work/cell_segmentation/Usiigaci/Mask R-CNN/mrcnn/utils.py", line 514, in resize_mask
    mask = scipy.ndimage.zoom(mask, zoom=[scale, scale, 1], order=0)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/scipy/ndimage/interpolation.py", line 615, in zoom
    shape=output_shape)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/site-packages/scipy/ndimage/_ni_support.py", line 7

image_info {'id': 41, 'source': 'custom', 'path': './sample_dataset/images/sample_6.jpg', 'annotations': ['./sample_dataset/annotations/sample_6/cell.png', './sample_dataset/annotations/sample_6/boundary.png']}
class_id {'': 0, 'boundary': 1, 'cell': 2}


Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-13:
Process ForkPoolWorker-5:
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/pool.py", line 589, in _terminate_pool
    p.terminate()
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/process.py", line 116, in terminate
    self._popen.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/joanna/anaconda3/envs/usiigaci/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/joanna/anaconda3/envs/usiigaci/lib/p

KeyboardInterrupt: 

## Evaluation

Run evaluation on chosen model and visualize predictions.